In [1]:
import tensorflow as tf
import numpy as np

In [30]:
#List out our bandits. Currently bandit 4 (index#3) is set to most often provide a positive reward.
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [3]:
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [8]:
init = tf.initialize_all_variables()
weights = tf.random_normal([4])
responsible_weight = tf.slice(weights,[1],[1])

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    #print(sess.run(weights))
    ww, resp = sess.run([weights, responsible_weight])
    print(resp)
    print(ww)
    #print(ww)

[ 1.43618107]
[-0.50469244  1.43618107 -0.95909572  0.03621461]


In [218]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.initialize_all_variables()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.
        
        #Update the network.
        cost,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 50 == 0:
            print cost, resp, ww
            print "Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward)
        i+=1
        
print "The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising...."
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print "...and it was right!"
else:
    print "...and it was wrong!"

None [ 1.] [ 1.00100005  1.          1.          1.        ]
Running reward for the 4 bandits: [ 1.  0.  0.  0.]
None [ 1.03828609] [ 0.99900109  0.99900103  0.99900204  1.03924918]
Running reward for the 4 bandits: [ -1.  -1.  -1.  40.]
None [ 1.08447266] [ 0.99800009  0.99900103  0.99900204  1.08539474]
Running reward for the 4 bandits: [ -2.  -1.  -1.  89.]
None [ 1.12699616] [ 0.99699807  0.99900103  1.00100303  1.12788343]
Running reward for the 4 bandits: [  -3.   -1.    1.  136.]
None [ 1.16882682] [ 0.99599504  1.00000203  1.00100303  1.16968238]
Running reward for the 4 bandits: [  -4.    0.    1.  184.]
None [ 1.20838273] [ 0.99599606  1.00100207  1.00100303  1.20921028]
Running reward for the 4 bandits: [  -4.    1.    1.  231.]
None [ 1.24668372] [ 0.99499202  1.00200105  1.00000405  1.24748588]
Running reward for the 4 bandits: [  -5.    2.    0.  278.]
None [ 1.28539896] [ 0.99499202  1.00200105  0.99900407  1.28617692]
Running reward for the 4 bandits: [  -5.    2.   -1.

In [109]:
np.argmax(ww)

3

In [110]:
np.argmax(-np.array(bandits))

3

In [112]:
-np.array(bandits)

array([-0.2, -0. ,  0.2,  5. ])